In [26]:
from dotenv import load_dotenv
import os
import getpass

import google.generativeai as genai
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory
from langchain.memory import ConversationBufferWindowMemory
from langchain.memory import ConversationTokenBufferMemory
from langchain.memory import ConversationSummaryBufferMemory

load_dotenv()

False

In [3]:
if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Provide your Google API Key")

Provide your Google API Key··········


# Introduction

**Langchain** is a framework for developing applications powered by large language model. It is an open source ochestration framework for the development of applications that uses LLM

It is a generic interface for any LLM, abstracting the complexities of interacting with different LLMs

Chains, in langchain, are core of the langchain work flow. It combine LLMs with other components, creating applications by executing a sequence of functions

### Chat Models

**Chat models** is a language model that uses chat messages as input and return chat messages as outputs.

In [5]:
llm = ChatGoogleGenerativeAI(model="gemini-pro")
result = llm.invoke("What is 2 + 2?")
print(f"Response: {result.content}")

Response: 4


## Prompt Template
It takes a prompt as input, which is set of instructions or input provided by user to guie model's response. It helps generate relevant output and understanding the context

**Prompt template** class formalizes the composition of prompts without the need to manually hard code context and queries

In [10]:
template = """
Translate the text delimited by triple back ticks into
style that is {style}

text: ```{text}```
"""

In [13]:
prompt_template = ChatPromptTemplate.from_template(template)
print(f"Prompt template passed to the model:\n{prompt_template.messages[0].prompt.template}")

Prompt template passed to the model:
 
Translate the text delimited by triple back ticks into
style that is {style}

text: ```{text}```



In [15]:
print(f"Inputs in the prompt: {prompt_template.messages[0].prompt.input_variables}")

Inputs in the prompt: ['style', 'text']


In [16]:
customer_email = "Yeh kya hae bhai? Yeh woh cheez nahi hae jo mein nein \
order ki thi. Tasweer mein yeh buhat muhktalif tha. Abb mein kya karoon? \
Replacment or return ki kya policy hae?"

style = "American style that is polite, calm and respectful in tone"

In [18]:
customer_message = prompt_template.format_messages(style=style,
                                                   text=customer_email)

print(f"Customer message: {customer_message[0]}")

Customer message: content=' \nTranslate the text delimited by triple back ticks into\nstyle that is American style that is polite, calm and respectful in tone\n\ntext: ```Yeh kya hae bhai? Yeh woh cheez nahi hae jo mein nein order ki thi. Tasweer mein yeh buhat muhktalif tha. Abb mein kya karoon? Replacment or return ki kya policy hae?```\n'


In [20]:
response = llm.invoke(customer_message)
print(f"Model response: {response.content}")

Model response: Excuse me, this is not what I ordered. It looks very different from the picture. What should I do now? What is your policy on replacements or returns?


In [24]:
service_reply = """
Thanks for reaching out. We are sorry for inconvenience. \
We upload high quality pictures that best reflects the \
product quality. You can replace the item if it is ordered \
within 7 days. You cannot return the item. Our team will check \
the item and tell you what to do next. Thanks for understanding
"""

service_language = """
Translate the text into Urdu words using polite
tone.
"""

service_message = prompt_template.format_messages(style=service_language,
                                                  text=service_reply)


service_response = llm.invoke(service_message)
print(service_response.content)

شکریہ کہ ہمارے پاس پہنچے۔ ہم آپ کو ہونے والی پریشانی پر معذرت خواہ ہیں۔ ہم بہترین معیار کی تصاویر اپ لوڈ کرتے ہیں جو مصنوعات کے معیار کی عکاسی کرتی ہیں۔ اگر آپ نے 7 دنوں کے اندر آرڈر کیا ہے تو آپ اس آئٹم کو تبدیل کر سکتے ہیں۔ آپ اس آئٹم کو واپس نہیں کر سکتے۔ ہماری ٹیم آئٹم چیک کرے گی اور آپ کو بتائے گی کہ اس کے بعد کیا کرنا ہے۔ سمجھنے کے لئے شکریہ


You can see from the above example that instead of creating template for each response, we can create a template and reuse it

## Memory
Traditionally, large language models (LLMS) doesn't have any long term memory of prior conversation unless you pass the whole chat history as an input. Langchain solve this problem with simple utilities for adding in memory into your application

When we alk about memory in langchain, we are referring to how the system remember the information from previous interactions

In [25]:
user = "Hello! My name is Maaz"
response = llm.invoke(user)
print(f"User: {user}\nResponse: {response.content}")

user = "What is 2 + 2?"
response = llm.invoke(user)
print(f"User: {user}\nResponse: {response.content}")

user = "What is my name?"
response = llm.invoke(user)
print(f"User: {user}\nResponse: {response.content}")

User: Hello! My name is Maaz
Response: Hello Maaz! It's great to meet you. I am Gemini, a multimodal AI language model. I am designed to understand and generate human language, and to answer your questions and assist you with a wide range of tasks. Is there anything I can help you with today?
User: What is 2 + 2?
Response: 4
User: What is my name?
Response: I do not have access to your personal information, including your name, as I am a chatbot assistant and do not have a physical presence or the ability to interact with the real world.


We can see that the model didn't remember my name despite being recently shared. The response of LLM model is stateless

### Conversation Buffer Memory

It stores the entire conversation history from beginning to end

In [28]:
from langchain.memory import ConversationBufferMemory

llm = llm = ChatGoogleGenerativeAI(model="gemini-pro")

memory = ConversationBufferMemory()
conversation = ConversationChain(
    llm=llm,
    memory = memory,
    verbose=True
)

conversation.predict(input = "My name is Maaz")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: My name is Maaz
AI:

> Finished chain.


'Hello Maaz, my name is Gemini. How can I help you today?'

In [30]:
conversation.predict(input="What is 2 + 2?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: My name is Maaz
AI: Hello Maaz, my name is Gemini. How can I help you today?
Human: What is 2 + 2?
AI:

> Finished chain.


'2 + 2 is 4.'

In [31]:
conversation.predict(input="What is my name?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: My name is Maaz
AI: Hello Maaz, my name is Gemini. How can I help you today?
Human: What is 2 + 2?
AI: 2 + 2 is 4.
Human: What is my name?
AI:

> Finished chain.


'Your name is Maaz, as you told me earlier.'

In [33]:
print(f"Entire conversation: \n{memory.buffer}")

Entire conversation: 
Human: My name is Maaz
AI: Hello Maaz, my name is Gemini. How can I help you today?
Human: What is 2 + 2?
AI: 2 + 2 is 4.
Human: What is my name?
AI: Your name is Maaz, as you told me earlier.


In [36]:
memory.load_memory_variables({})

{'history': 'Human: My name is Maaz\nAI: Hello Maaz, my name is Gemini. How can I help you today?\nHuman: What is 2 + 2?\nAI: 2 + 2 is 4.\nHuman: What is my name?\nAI: Your name is Maaz, as you told me earlier.'}

### Conversation Buffer Window Memory

It stores only the most recent exchanges within defined window size. Instead of saving entire conversation, it focuses on recent conversation

It is useful when the goal is to make AI focus on recent chat without overwheling by too much information

In [39]:
from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(k=1) #remember only 1 recent message

memory.save_context({"input": "Hi"},
                    {"output": "What's up"})
memory.save_context({"input": "Not much, just hanging"},
                    {"output": "Cool"})

print(f"Conversations: \n{memory.buffer}")

Conversations: 
Human: Not much, just hanging
AI: Cool


In [41]:

memory = ConversationBufferWindowMemory(k=2) #remember only 2 recent message

memory.save_context({"input": "Hi"},
                    {"output": "What's up"})
memory.save_context({"input": "Not much, just hanging"},
                    {"output": "Cool"})
memory.save_context({"input": "How is the weather"},
                    {"output": "Hot"})

print(f"Conversations: \n{memory.buffer}")

Conversations: 
Human: Not much, just hanging
AI: Cool
Human: How is the weather
AI: Hot


### Conversation Token Buffer Memory

Similar to the conversation buffer memory but instead of counting interactions, it count tokens

This memory type limits how much the AI remembers based on number of tokens. Once the token limit is reached, the oldest tokens are forgotten

In [45]:
from langchain.memory import ConversationTokenBufferMemory

memory = ConversationTokenBufferMemory(llm=llm, max_token_limit=40)
memory.save_context({"input": "AI is what?!"},
                    {"output": "Amazing!"})
memory.save_context({"input": "Backpropagation is what?"},
                    {"output": "Beautiful!"})
memory.save_context({"input": "Chatbots are what?"},
                    {"output": "Charming!"})

print(f"Conversations: {memory.buffer}")

Conversations: Human: AI is what?!
AI: Amazing!
Human: Backpropagation is what?
AI: Beautiful!
Human: Chatbots are what?
AI: Charming!


In [47]:
memory.save_context({"input": "How is the weather?"},
                    {"output": "Windy and cold!"})
print(f"Memory: \n{memory.buffer}")

Memory: 
AI: Beautiful!
Human: Chatbots are what?
AI: Charming!
Human: How is the weather?
AI: Windy!
Human: How is the weather?
AI: Windy and cold!


### Conversation Summary Memory

It doesn't store every detail but creates a summary of past interactions. This allows the model to remember what is important without needing full conversation

In [50]:
from langchain.memory import ConversationSummaryBufferMemory

schedule = "There is a meeting at 8am with your product team. \
You will need your powerpoint presentation prepared. \
9am-12pm have time to work on your LangChain \
project which will go quickly because Langchain is such a powerful tool. \
At Noon, lunch at the italian resturant with a customer who is driving \
from over an hour away to meet you to understand the latest in AI. \
Be sure to bring your laptop to show the latest LLM demo."

memory = ConversationSummaryBufferMemory(llm=llm, max_token_limit=100)
memory.save_context({"input": "Hello"}, {"output": "What's up"})
memory.save_context({"input": "Not much, just hanging"},
                    {"output": "Cool"})
memory.save_context({"input": "What is on the schedule today?"},
                    {"output": f"{schedule}"})

print(f"Conversation in memory: \n{memory.buffer}")

Conversation in memory: 
System: The human says hello. The AI asks what's up. The human says they are hanging. The AI says that's cool. The human asks what is on the schedule today.
AI: There is a meeting at 8am with your product team. You will need your powerpoint presentation prepared. 9am-12pm have time to work on your LangChain project which will go quickly because Langchain is such a powerful tool. At Noon, lunch at the italian resturant with a customer who is driving from over an hour away to meet you to understand the latest in AI. Be sure to bring your laptop to show the latest LLM demo.


In [51]:
conversation = ConversationChain(llm=llm,
                                 memory=memory,
                                 verbose=True)

conversation.predict(input="What would be a good demo to show?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
System: The human says hello. The AI asks what's up. The human says they are hanging. The AI says that's cool. The human asks what is on the schedule today.
AI: There is a meeting at 8am with your product team. You will need your powerpoint presentation prepared. 9am-12pm have time to work on your LangChain project which will go quickly because Langchain is such a powerful tool. At Noon, lunch at the italian resturant with a customer who is driving from over an hour away to meet you to understand the latest in AI. Be sure to bring your laptop to show the latest LLM demo.
Human: What would be a good demo to show?
AI:

> Finished chain.


'I recommend the latest demo for LLM that allows the model to generate text in the voice of a specific person, including celebrities.'

In [52]:
print(f"New memory: {memory.buffer}")

New memory: System: The human says hello. The AI asks what's up. The human says they are hanging. The AI says that's cool. The human asks what is on the schedule today. The AI says there is a meeting at 8am with the product team, work on the LangChain project from 9am-12pm, and lunch at an Italian restaurant with a customer at Noon.
Human: What would be a good demo to show?
AI: I recommend the latest demo for LLM that allows the model to generate text in the voice of a specific person, including celebrities.
